In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

In [5]:
# Load the dataset
(train_ds, test_ds), ds_info = tfds.load(
    "plant_village",
    split=["train[:80%]", "train[80%:]"],
    as_supervised=True,
    with_info=True
)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/plant_village/incomplete.X10220_1.0.2/plant_village-train.tfrecord*...:   …

Dataset plant_village downloaded and prepared to /root/tensorflow_datasets/plant_village/1.0.2. Subsequent calls will reuse this data.


In [7]:
# Get number of classes
num_classes = ds_info.features['label'].num_classes
class_names = ds_info.features['label'].names

print(f"Number of classes: {num_classes}")

Number of classes: 38


In [8]:
IMG_SIZE = 227
BATCH_SIZE = 32

def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0  # Normalize

    return image, label

train_ds = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [10]:
model = Sequential([

    # First convolutional layer: Applies 96 filters of size 11x11 with stride (4,4)
    # Activation function ReLU adds non-linearity
    # Input shape for the image is 227x227 with 3 color channels (RGB)
    Conv2D(96, (11, 11), strides=(4, 4), activation='relu', input_shape=(227, 227, 3)),

    # MaxPooling layer to reduce spatial dimensions after the first convolution
    # Pooling size is 3x3 and strides are 2x2 to downsample the feature map
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    # Second convolutional layer: Applies 256 filters of size 5x5 with 'same' padding (preserves dimensions)
    # Activation function is ReLU
    Conv2D(256, (5, 5), padding='same', activation='relu'),

    # MaxPooling layer to reduce spatial dimensions after the second convolution
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    # Third convolutional layer: Applies 384 filters of size 3x3 with 'same' padding
    # Activation function is ReLU
    Conv2D(384, (3, 3), padding='same', activation='relu'),

    # Fourth convolutional layer: Similar to the third, with 384 filters of size 3x3 and 'same' padding
    Conv2D(384, (3, 3), padding='same', activation='relu'),

    # Fifth convolutional layer: Applies 256 filters of size 3x3 with 'same' padding
    # Activation function is ReLU
    Conv2D(256, (3, 3), padding='same', activation='relu'),

    # MaxPooling layer to reduce spatial dimensions after the convolutional layers
    MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),

    # Flattening layer: Converts the 2D output from the last convolutional layer into 1D vector
    Flatten(),

    # First fully connected (Dense) layer with 4096 neurons and ReLU activation
    Dense(4096, activation='relu'),

    # Dropout layer to prevent overfitting by randomly setting 50% of the neurons to zero during training
    Dropout(0.5),

    # Second fully connected (Dense) layer with 4096 neurons and ReLU activation
    Dense(4096, activation='relu'),

    # Another Dropout layer for regularization
    Dropout(0.5),

    # Output layer: `num_classes` neurons for multi-class classification, using Softmax activation
    # Softmax converts the outputs to probability values that sum to 1
    Dense(num_classes, activation='softmax')

])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
#compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [12]:
#model summary
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_10 (Conv2D)              │ (None, 55, 55, 96)     │        34,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 27, 27, 96)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 27, 27, 256)    │       614,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 13, 13, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 13, 13, 384)    │       885,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 13, 13, 384)    │     1,327,488 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_14 (Conv2D)              │ (None, 13, 13, 256)    │       884,992 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 6, 6, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 9216)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 4096)           │    37,752,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 4096)           │    16,781,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 38)             │       155,686 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 58,437,030 (222.92 MB)

 Trainable params: 58,437,030 (222.92 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#Train the model
history = model.fit(train_ds,validation_data=test_ds,epochs=10)